In [1]:
data_path = r"dmv_site_data_base" #the dataset version
question_path = r"gpt-4_test_questions.json"

In [2]:
import redis

# Basic connection to Redis Stack running locally
client = redis.Redis(host='127.0.0.1', port=6379, decode_responses=True)

# Test the connection
try:
    response = client.ping()
    print("Connection to Redis successful!")
    print("Redis PING response:", response)
except redis.exceptions.ConnectionError as e:
    print(f"Connection to Redis failed: {e}")

Connection to Redis successful!
Redis PING response: True


In [3]:
import numpy as np
from redis.commands.search.query import Query
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path="c:/Users/benja/startup_projects/civgen/.env")
openai_client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

# 1. Encode query (keep this the same)
query_text = "How do I register a low powered bike?"
query_vector = np.array(
    openai_client.embeddings.create(
        input=query_text,
        model="text-embedding-3-large"
    ).data[0].embedding, 
    dtype=np.float32
)

# 2. Update query to match schema fields
query = (
    Query('(*)=>[KNN 2 @vector $query_vector AS vector_score]')
    .sort_by('vector_score')
    .return_fields(
        'vector_score', 
        'url',         # From TextField("$.url")
        'title',       # From TextField("$.title")
        'text_content',# From TextField("$.text_content")
        'attachment_paths',  # From TextField("$.attachments.*")
        'page'
    )
    .dialect(2)
)

# 3. Execute search (keep the same)
result = client.ft(f'idx:{data_path}_vss').search(
    query,
    {'query_vector': query_vector.tobytes()}
)

# 4. Process results with schema-aligned fields
for doc in result.docs:
    print(f"""
    Score: {doc.vector_score}
    URL: {doc.url}
    Title: {doc.title}
    Content: {doc.text_content}
    Attachments: {doc.attachment_paths}
    Page: {doc.page}
    """)



    Score: 0.536603569984
    URL: https://www.dmv.virginia.gov/vehicles/registration/moped
    Title: All About Mopeds
    Content: A moped is a small motorized vehicle that can be powered by pedals or an engine. Different rules apply to mopeds than bicycles or motorcycles.
Moped Rules of the Road
You must be at least age 16 to drive a moped.
Every moped driver operating on Virginia roadways must carry a government-issued photo ID (does not have to be a driver's license).
Every driver and passenger must wear a Virginia State Police-approved helmet while riding a moped. Drivers must also wear a face shield, safety glasses or goggles unless the moped has a windshield.
It is illegal to drive a mopeds on the interstate.
It is illegal to drive a moped if your license is suspended or revoked for convictions of DUI, underage consumption of alcohol, refusing a blood/breath test, or driving while suspended/revoked for a DUI-related offense.
If your driving privilege is suspended or revoked fo